# Blur and Clear NN Notebook
https://github.com/aditya9211/Blur-and-Clear-Classification

### General Quick Start Steps
1. Clone repo https://github.com/aditya9211/Blur-and-Clear-Classification.git
2. install dependencies- recommend creating a conda env first
    a. create a conda enivironment with a yml file `conda env create -f BlurAndClear.yml` 
    b. `pip install -r requirements.txt`
3. Train the network -  train.py with folders with good and bad example data
4. Test the network - test.py
5. Predict Output - predict.py --img imagename.jpg   THIS NEEDS WORK

In [2]:
import os
os.chdir('F:/CACO_HOM/Blur-and-Clear-Classification/') #Change to wherever you cloned the repo to
print(os.getcwd())

F:\CACO_HOM\Blur-and-Clear-Classification


Install dependencies using one of the methods in quick start step 2 then proceed

In [ ]:
#The yml file should have the following organization
"""
name: BlurAndClear
channels:
    - conda-forge
    - defualts
dependencies:
    - python = 3.5
"""

In [3]:
from __future__ import print_function
import argparse
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.misc as ms
import scipy.ndimage as nd
import matplotlib.pyplot as plt
import glob
import shutil
#These are helper functions in train/test and predict - located in directory
from utils import path_validation, resize, sigmoid, h, validate, model_score     

These are all of the hyper-parameters and lists of constants in the config.py script. These can be edited here or uncomment the box below `from config import *`

In [4]:
# Stores current executing path
HOME_FOLDER_PATH = os.getcwd()
print(os.getcwd())

# if not passes default model path
MODEL_PATH = HOME_FOLDER_PATH + '/model/result.pkl'
DATA_PATH = HOME_FOLDER_PATH + '/data'

# Path where splitted data get stored
TRAIN_DATA_PATH = DATA_PATH + '/train_images.npy'
TRAIN_LABEL_PATH = DATA_PATH + '/train_labels.npy'
TEST_DATA_PATH = DATA_PATH + '/test_images.npy'
TEST_LABEL_PATH = DATA_PATH + '/test_labels.npy'

# Path for saving plot of cost vs iterations
PLOT_PATH = DATA_PATH + '/loss_decay.png'


# Median Filter size
radius = 3

# width and height of resized image
width = 100
height = 100

# Splitting ratio for training & testing
split_ratio = 0.4

# Seed to get same results on re-run
seed = 10

# Size of data to be fed at each epochs
batch_size = 10

# Hidden Layer neurons size
Neurons_size = 300

# Iteration in NN Model
max_iter = 50

# Logging steps to show summary
logging_steps = min(1, max_iter)

# Learning rate in NN Model
alpha = 0.001

# Regularization Term used in cost
Lambda = 0.0007

# Default Activation Function
act = 'sig'

F:\CACO_HOM\Blur-and-Clear-Classification


In [ ]:
#from config import *   #These are parameters in the previous section

### Train the Network
The following section is equivalent to running the following script inside this directory, which you could do by uncommenting and running the cell below - the script would also require there to be two folders of good and bad training data and to give their location. 
Example: `example: python train.py --good_path F:\CACO_HOM\Rectified_imagery\good --bad_path F:\CACO_HOM\Rectified_imagery\bad`

Or - import the functions in the subsequent box and then you can run the main model function - noting that you will have to change the IMG_PATH manually rather than giving it as an argument. To see the meaning of the functions see the script.

The script is for pre-processing the data by resizing, median filtering the images before training the Neural Network model for the task of classifying blur and clear images.

In [ ]:
# !python train.py --good_path 'path' --bad_path 'path'  #See the .py scrip for all the parameter definitions

In [ ]:
from train import save_data, NN_Model, derivative, cost, back_propagate, get_batch, output_encoding, show_plot

In [ ]:
%%time
print ('Pre-Processsing the Data...........\n')

GOOD_IMG_PATH = 'F:/CACO_HOM/Rectified_imagery/good/'
BAD_IMG_PATH = 'F:/CACO_HOM/Rectified_imagery/bad/'
    
# Reading the Good Images 
good_img = []
for filename in os.listdir(GOOD_IMG_PATH):
    good_img.append(ms.imread(GOOD_IMG_PATH+filename, mode='L'))
good_img = np.asarray(good_img)

# Reading the Bad Images 
bad_img = []
for filename in os.listdir(BAD_IMG_PATH):
    bad_img.append(ms.imread(BAD_IMG_PATH+filename, mode='L'))
bad_img = np.asarray(bad_img)

# Concatenate the array of Good & Bad images
combined_img = np.concatenate((good_img, bad_img))  
labels = np.concatenate((np.ones(good_img.shape[0]), 
                        np.zeros(bad_img.shape[0])))
 
# Filtering the combined images to Reduce the Noise present
combined_img = nd.median_filter(combined_img, radius)

#__________________________________________________________________________________


def main():
    """
    Pre-process the data with filtering, resizing 
    and trained the Neural Networks with 
    resulting pre-processed data using backpropagation

    """

    ## Input Layer -> 10001 U
    ## 1 Hidden Layers -> 300 HU 
    ## 1 Output Layer -> 2 Neurons

    # Path Validation
    if not path_validation(GOOD_IMG_PATH, read_access=True):
        exit(0)
    if not path_validation(BAD_IMG_PATH, read_access=True):
        exit(0)

    # Model Path Vaildation
    if path_validation(MODEL_PATH):
        print ('\nModel Path Success .....\n')


    # Getting the Same Result in Shuffle in each Run.
    np.random.seed(SEED)

    # Convert the Good & Bad Images to Cumulative numpy array 
    #imgs, labels = data_preprocess(GOOD_IMG_PATH, BAD_IMG_PATH, radius=RADIUS)
    imgs = combined_img
    
    # Resizing the feature space for easier to handle
    imgs = resize(imgs, width=WIDTH, height=HEIGHT)

    # Splitting the Data for Training and Testing Purpose
    print('\nSplitting of Data......\n')
    train_images, test_images, train_labels, test_labels = train_test_split(imgs, labels, 
                                                        test_size=SPLIT_RATIO, random_state = SEED) 

    # Saving the splitted data to disk
    save_data(train_images, train_labels, test_images, test_labels)

    # No of unique class in data
    nclass = np.unique(labels).shape[0]
    			
    # Addition of Bias in Train/Test Images
    train_images = np.insert(train_images, 0, 1, axis=1) 
    test_images = np.insert(test_images, 0, 1, axis=1)

    # May Used for Cal No Of Neuron as hyper-parameters to Good value
    no_of_neurons = train_images.shape[0]/(2*(train_images.shape[1]+10))


    # Intializing the Model
    theta = NN_Model([train_images.shape[1],NEURONS_SIZE,nclass])

    print ("BAckPROP .................\n")
    params = back_propagate(theta['Theta1'], theta['Theta2'], train_images, train_labels,
                            nclass, alpha=ALPHA, lambdaa=LAMBDA, max_iter=MAX_ITER, act=ACT, 
                            batch_size=BATCH_SIZE, logging=LOGGING_STEPS)

    # Accuracy Score on Train set
    accuracy = model_score(params, train_images, train_labels, act=ACT) 
    print('\nAccuracy on Train Data: ', accuracy)

    # Accuracy Score on test set
    accuracy = model_score(params, test_images, test_labels, act=ACT) 
    print('\nAccuracy on Test Data: ', accuracy)

    # Storing the Results in tmp directory 
    print ('\nSaving Results...............\n')
    joblib.dump(params, MODEL_PATH)
 
    # Plotting the Curve
    show_plot(params['Loss'], PLOT_PATH)
    
#________________________________________________________

if __name__ == "__main__":
    main()

print ('finished')

### Test the Network
Test the neural network with test data stored by train.py

In [5]:
params = joblib.load(MODEL_PATH)
test_images = np.load(TEST_DATA_PATH)
test_labels = np.load(TEST_LABEL_PATH)

# Addition of bias in test set
test_images = np.insert(test_images, 0, 1, axis=1)

accuracy = model_score(params, test_images, test_labels, act='sig')
print ('\nAccuracy : ' + str(accuracy) + ' %\n')


Accuracy : 86.60714285714286 %



### Predict Part
predict the label of images(Good/Bad) provided by argument while calling

In [20]:
%%time

#output image locations
bad_p = 'F:\\CACO_HOM\\predicted_bad\\'
good_p = 'F:\\CACO_HOM\\predicted_good\\'
    
#images to predict
predict_path = 'F:\\CACO_HOM\\Rectified_imagery\\predict\\'
filenames = sorted(glob.glob(predict_path + '1576*.jpg'))
#files = sorted(os.listdir(predict_path))

for i in filenames:
    # Reading images in grayscale mode
    img = ms.imread(i, mode='L')
    # Applying median filter to remove noise
    img = nd.median_filter(img, radius)
    # To make it 2D
    img = img[np.newaxis, :]
    # Resizing the images to that of train
    img = resize(img, width=width, height=height)
    # Addition of bias term
    img = np.insert(img, 0, 1, axis=1)
    
    # preprocess the images
    #img = predict_preprocess(i)
    
    ## Find the label predicted by the model
    predicted_label = validate(params, img)
    
    files = sorted(os.listdir(predict_path))
    #for label in predicted_label:
        #for f in files:
            #if label:
                #print(f + " is a Good Image\n")
                #shutil.move(predict_path+f, good_p + f)
            #else:
                #print(f+ " is a Bad Image\n")
            #shutil.move(predict_path+f, bad_p + f)
    
    for label in predicted_label:
        if label:
            print(i + " is a Good Image\n")
            shutil.copy(i, good_p + i[38:52])
        else:
            print(i + " is a Bad Image\n")
        shutil.copy(i, bad_p + i[38:52])


c:\users\jover\anaconda3\envs\blurandclear\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':



Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576267201.jpg is a Good Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576269001.jpg is a Good Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576270801.jpg is a Good Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576512001.jpg is a Bad Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576524601.jpg is a Good Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576526401.jpg is a Good Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576528201.jpg is a Good Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576530001.jpg is a Bad Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576584001.jpg is a Bad Image


Resizing the images .........

F:\CACO_HOM\Rectified_imagery\predict\1576596601.jpg 

In [8]:
print(filenames)

[]
